In [1]:
import h5py
import numpy as np
import pandas as pd
import glob

In [ ]:
 f = h5py.File('data/delphes_data4320.h5', 'r')

In [ ]:
f.keys()

In [ ]:
f["sample_summary"]["signal_events_per_benchmark"].value

In [ ]:
path = "./data/"
delphesDatasetList = [f for f in glob.glob(path + "delphes_data?.h5")]
delphesDatasetList += [f for f in glob.glob(path + "delphes_data??.h5")]
delphesDatasetList += [f for f in glob.glob(path + "delphes_data???.h5")]
delphesDatasetList += [f for f in glob.glob(path + "delphes_data????.h5")]

In [ ]:
nEventsList = []
for fname in delphesDatasetList:
    with h5py.File(fname, 'r') as f:
        n_events = f["sample_summary"]["signal_events_per_benchmark"].value
        nEventsList.append(n_events)
nEventsList = np.array(nEventsList)

In [ ]:
kfactorList = nEventsList/nEventsList.sum(axis=0).astype(float)
#kfactorList = np.nan_to_num(kfactorList)

In [ ]:
kfactorList.shape

In [ ]:
kfactorList[kfactorList == 0] = np.nan
means, std = np.nanmean(kfactorList, axis=0), np.nanstd(kfactorList, axis=0)

In [ ]:
means # array([0.00033636, 0.00033625, 0.00029516, 0.00240964, 0.00033625,0.00240964])

In [ ]:
std

In [ ]:
np.save("kfactorList.npy",kfactorList)

# Alternate better k factor calculation

In [ ]:
kfactorList = np.load("kfactorList.npy")

In [ ]:
n_files_benchmark = (~np.isnan(kfactorList)).sum(axis=0)

In [ ]:
kfac = 1./n_files_benchmark

In [ ]:
kfac # array([0.00033636, 0.00033625, 0.00029516, 0.00240964, 0.00033625,0.00240964])

# Load combined dataset and multiply mean k factors

In [2]:
 #f = h5py.File('data/delphes_data_shuffled.h5', 'r+') #careful!!!
    f = h5py.File('data/delphes_data_shuffled.h5', 'r')

In [ ]:
k_factors = np.array([0.00033636, 0.00033625, 0.00029516, 0.00240964, 0.00033625,0.00240964])
n_benchamrks = len(k_factors)

In [ ]:
f.keys()

In [ ]:
f["samples"].keys()

In [3]:
sampling_benchmarks = f["samples"]["sampling_benchmarks"].value
weights = f["samples"]["weights"].value
oldWeights = np.copy(weights)

/usr/local/lib/python2.7/dist-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [ ]:
for i in range(n_benchamrks):
    weights[sampling_benchmarks == i] *= k_factors[i]

In [4]:
weights.mean(), oldWeights.mean() # (4.952211578590396e-07, 4.959792504458865e-10)

(2.2659419453842956e-10, 2.2659419453842956e-10)

In [ ]:
fileWeights = f["samples"]["weights"]
fileWeights[...] = weights 
f.close()

# Tests

In [ ]:
a = np.arange(600).reshape(100,6); b = np.arange(6)

In [ ]:
a.shape, b.shape

In [ ]:
weights.shape, kfactors.shape

In [ ]:
(a*b).shape

In [ ]:
a[:3]

In [ ]:
(a*b)[:3]